In [ ]:
from pyspark.sql import Row, SparkSession

# List all databases
databases = spark.sql("SHOW DATABASES").collect()

# List to hold specific table metadata
table_metadata = []

for db in databases:
    db_name = db.databaseName
    tables = spark.sql(f"SHOW TABLES IN `{db_name}`").collect()

    for table in tables:
        table_name = table.tableName

        try:
            # Use DESCRIBE FORMATTED to get the table type (Managed or External)
            describe_formatted_result = spark.sql(f"DESCRIBE FORMATTED `{db_name}`.`{table_name}`").collect()
            table_type = next((row.data_type.split(':', 1)[1].strip() for row in describe_formatted_result if 'Table Type' in row.col_name), 'Unknown')

            table_metadata.append({'schema_name': db_name, 'table_name': table_name, 'table_type': table_type})

        except Exception as e:
            print(f"Error processing table {db_name}.{table_name}: {e}")

# Convert list of metadata to DataFrame
metadata_df = spark.createDataFrame([Row(**metadata) for metadata in table_metadata])

# Save the DataFrame as a table named 'inventory'
metadata_df.write.mode("overwrite").saveAsTable("inventory")
